In [70]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()

from functools import partial

from IPython.display import display

In [71]:
'''from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.cluster import IGraphLabelCooccurenceClusterer
from skmultilearn.ensemble import LabelSpacePartitioningClassifier'''

'from sklearn.ensemble import RandomForestClassifier\nfrom skmultilearn.problem_transform import LabelPowerset\nfrom skmultilearn.cluster import IGraphLabelCooccurenceClusterer\nfrom skmultilearn.ensemble import LabelSpacePartitioningClassifier'

In [72]:
aisles_df = pd.read_csv('aisles.csv')
departments_df = pd.read_csv('departments.csv')
order_products_prior_df = pd.read_csv('order_products__prior.csv')
order_products_train_df = pd.read_csv('order_products__train.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')

In [73]:
print('Total aisles: {}'.format(aisles_df.shape[0]))
display(aisles_df.head())
print('Total departments: {}'.format(departments_df.shape[0]))
display(departments_df.head())
print('Total orders: {}'.format(orders_df.shape[0]))
display(orders_df.head())
print('Total products: {}'.format(products_df.shape[0]))
display(products_df.head())

Total aisles: 134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


Total departments: 21


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


Total orders: 3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


Total products: 49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [74]:
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [75]:
order_products_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [76]:
test_df = pd.read_csv('sample_submission.csv')
test_df.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [77]:
order_products_prior_df.shape[0]

32434489

In [78]:
order_products_train_df.shape[0]

1384617

In [79]:
test_df.shape[0]

75000

In [80]:
orders_df[orders_df['order_id']==17]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
611292,17,36855,test,5,6,15,1.0


In [81]:
orders_df[orders_df['user_id']==36855].sort_values('order_number')

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
611288,898818,36855,prior,1,1,13,NaN
611289,234692,36855,prior,2,0,9,30.0
611290,3248434,36855,prior,3,6,14,6.0
611291,1058761,36855,prior,4,5,13,6.0
611292,17,36855,test,5,6,15,1.0


In [82]:
test_df = test_df.drop('products', axis=1)

In [83]:
test_df = pd.merge(test_df,orders_df,how='left',on='order_id')
test_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,17,36855,test,5,6,15,1.0
1,34,35220,test,20,3,11,8.0
2,137,187107,test,9,2,19,30.0
3,182,115892,test,28,0,11,8.0
4,257,35581,test,9,6,23,5.0


In [84]:
train_df = pd.concat([order_products_train_df,order_products_prior_df]).reset_index(drop=True)
train_df.sort_values('order_id')

MemoryError: 

In [ ]:
train_df = pd.merge(train_df,orders_df,how='left',on='order_id')
train_df.head()

In [ ]:
test_df.head()

In [ ]:
test_df['user_id'].nunique()

In [ ]:
test_df = pd.merge(test_df, train_df[['user_id','product_id','add_to_cart_order']], how='left', on='user_id')
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df = test_df.drop('eval_set',axis=1)
test_df.head()

In [ ]:
train_df = train_df.drop('eval_set',axis=1)
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df[train_df['user_id']==36855]

In [ ]:
test_df.head()